### TOKENIZER


In [1]:
import pandas as pd 
import argparse
import pandas as pd
import os
import sentencepiece as spm
import codecs
import pickle

train_df = pd.read_csv(f"data/train.csv",encoding='utf-8')

train_dialect = train_df['dialect_form'].values.tolist()
train_standard = train_df['standard_form'].values.tolist()

In [4]:
# 모든 요소를 문자열로 변환
train_dialect_str = list(map(str, train_dialect))
train_standard_str = list(map(str, train_standard))

In [9]:
with codecs.open(f"tokenizer/bpe.train",'w','utf8') as fout:
    fout.write("\n".join(train_dialect_str + train_standard_str))

In [10]:
def train_bpe(fpath, vocab_size=4000):
    dir = os.path.dirname(fpath)
    train = f'--input={fpath} \
              --normalization_rule_name=identity \
              --model_prefix={dir}/bpe_{vocab_size} \
              --character_coverage=1.0 \
              --vocab_size={vocab_size} \
              --model_type=bpe \
              --unk_id=0 \
              --bos_id=1 \
              --eos_id=2 \
              --pad_id=3 \
              --unk_piece=<unk> \
              --bos_piece=<bos> \
              --eos_piece=<eos> \
              --pad_piece=<pad>'
    spm.SentencePieceTrainer.Train(train)


train_bpe(f'tokenizer/bpe.train',vocab_size=8000)

In [12]:
sp = spm.SentencePieceProcessor()
sp.Load(f'tokenizer/bpe_8000.model')

# Save Vocab
vocab = {}
for id_ in range(sp.get_piece_size()):
    token = sp.id_to_piece(id_)
    vocab[id_] = token

with open(f'tokenizer/vocab_dict_8000.pickle','wb') as f:
    pickle.dump(vocab,f)